<a href="https://colab.research.google.com/github/marcory-hub/swift-yolo/blob/main/Swift_YOLO_192_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make Swift-YOLO model 192x192 px


This colab notebook is based on the colab notebooks on [ssmca-model-zoo](https://github.com/Seeed-Studio/sscma-model-zoo)

**Category:** Object Detection

**Algorithm:** [Swift-YOLO](configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py)

**Dataset:** Custom dataset from Google Drive (requires Gmail account)

---

## Setup Instructions

Follow these steps before running the notebook:

### Step 1: Enable GPU Acceleration

1. Click `Runtime` → `Change runtime type`
2. Select `GPU` in `Hardware accelerator`
3. Click `Save`

**Expected environment** (verify in first code cell output):
- GPU: T4 or higher
- Python 3.11.13

---

### Step 2: Copy dataset in COCO format

1. Make sure images and labels from your dataset have this folder structure with these exact names. (Augmen

```
🗂️ dataset
  🗂️ train
  _annotations.coco.json
  image1.jpg
  image2.jpg
  ...
  🗂️ valid
  _annotations.coco.json
  image3.jpg
  image4.jpg
  ...
```

2. Zip the dataset folder to a file names `dataset.zip`. On mac use `zip -r dataset.zip dataset -i '*.jpg' '*.txt' '*.yaml'` to exclude metadatafiles

3. Make in /content/drive/MyDrive the folder `yolo`. Copy the `dataset.zip` file to this folder, it is needed to make a callibration image set and your yolo model, fe `best.pt` to this folder. For the model you can use a custom name and adjust it in the options below.

4. Copy the `dataset.zip` file to the folder /content/drive/MyDrive/yolo.


---

### Step 4: Adjust Number of Classes

⚠️ **IMPORTANT: Update class count if your dataset has a different number of classes**

- **Default**: 4 classes
- **Action**: Search for all occurrences of `num_classes` or class count settings in the notebook and update them to match your dataset

**Example**: If your dataset has 5 classes, change `num_classes=4` to `num_classes=5` in all relevant code blocks.

---

## Ready to Run

After completing all setup steps above, you can run the notebook cells in order. The notebook will:
1. Download your dataset from google drive
2. Train the Swift-YOLO model
3. Export the model as `int8_vela.tflite` for Grove Vision AI V2


# Check PGU and Python version

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Check if GPU is available
!nvidia-smi

# Check Python version
!python --version

## Prerequisites
### Setup SSCMA
Clone the [repository](https://github.com/Seeed-Studio/ModelAssistant) and install the dependencies.

(you can ignore the ERROR)

In [ ]:
!pip install ethos-u-vela
!git clone https://github.com/Seeed-Studio/ModelAssistant.git -b 2.0.0  #clone the repo
%cd ModelAssistant
!. ./scripts/setup_colab.sh

### Download the pretrain model weights file

In [ ]:
%mkdir -p Swift-YOLO_192
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/animal/animal_detection.pth -O Swift-YOLO_192/pretrain.pth

### Download the dataset
and put it in the correct directory (ModelAssistant/Swift-YOLO_192)


In [ ]:
# Copy zipped dataset to colab and unzip the dataset
!scp '/content/drive/MyDrive/dataset.zip' '/content/dataset.zip'
!unzip '/content/dataset.zip' -d '/content/ModelAssistant/Swift-YOLO_192/'

Optional use a custom swift-yolo model

In [ ]:
!cp '/content/drive/MyDrive/best_coco_bbox_mAP_epoch_100.pth' 'Swift-YOLO_192/pretrain.pth'

## Train a model with SSCMA
All the training parameters are in the `config.py` file, you can change the parameters to train your own model.


- `data_root` - the datasets path.
- `epochs`- the train epochs. **we use 10 epochs as an example**.
- `batch` - the batch size.
- `height` - the image height.
- `width` - the image width.
- `load_from` - the pretrained model path.
- `num_classes` - the number of classes.

You can overwrite the parameters in the `config.py` file by using the `--cfg-options` argument.

In [ ]:
!sscma.train configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=3  \
    height=192 \
    width=192 \
    batch=256 \
    lr=0.16 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

# Zip and download the Swift-YOLO_192 model

In [ ]:
import os
import zipfile
from google.colab import files

# Define the paths
folder_path = '/content/ModelAssistant/Swift-YOLO_192'
zip_filename = 'Swift-YOLO_Models.zip'

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # List all items in the directory
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)

        # Only add to zip if it's a file (skips all subdirectories)
        if os.path.isfile(item_path):
            zipf.write(item_path, arcname=item)
            print(f"Added to zip: {item}")

# Automatically trigger the download to your computer
files.download(zip_filename)

# Deploy model on [SenseCraft AI platform](https://sensecraft.seeed.cc/ai/model)

1. Click `My Own Models`
2. Select `+ Add Model`
3. Follow the instructions
4. Download the int8_vela.tflite model